In [4]:
import pandas as pd
import plotly.express as px

In [5]:
df = pd.read_csv("compound_price_data.csv")
fig = px.line(df, x="Date", y="Price")

# template="plotly_dark"

fig.update_layout(
    xaxis = dict(
        tickmode = "array",
        tickvals = ['Mar 01, 2022', 'Feb 05, 2022', 'Jan 06, 2022', 'Dec 01, 2021',
                    'Nov 01, 2021', 'Oct 02, 2021', 'Sep 02, 2021', 'Aug 03, 2021',
                    'Jul 04, 2021', 'Jun 04, 2021'],
        ticktext = ['Mar 2022', 'Feb 2022', 'Jan 2022', 'Dec 2021',
                    'Nov 2021', 'Oct 2021', 'Sep 2021', 'Aug 2021',
                    'July 2021', 'June 2021']
    )
)

# # chart line color red
# fig.update_traces(line_color='#FF0000')

fig.show()

In [6]:
# df = pd.read_csv("uniswap_price_data.csv")
# fig = px.line(df, x="Date", y="Price")

# # template="plotly_dark"

# fig.update_layout(
#     xaxis = dict(
#         tickmode = "array",
#         tickvals = ['Mar 01, 2022', 'Feb 05, 2022', 'Jan 06, 2022', 'Dec 01, 2021',
#                     'Nov 01, 2021', 'Oct 02, 2021', 'Sep 02, 2021', 'Aug 03, 2021',
#                     'Jul 04, 2021', 'Jun 04, 2021'],
#         ticktext = ['Mar 2022', 'Feb 2022', 'Jan 2022', 'Dec 2021',
#                     'Nov 2021', 'Oct 2021', 'Sep 2021', 'Aug 2021',
#                     'July 2021', 'June 2021']
#     )
# )

# # chart line color red
# fig.update_traces(line_color='#FF0000')

# fig.show()

In [7]:
from nltk.corpus import stopwords
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/lamchingrou/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
def clean_content(contentInput):
    if type(contentInput) == np.float:
        return ""
    contentInput = contentInput.lower()
    contentInput = re.sub("'", "", contentInput) # to avoid removing contractions in english
    contentInput = re.sub("@[A-Za-z0-9_]+","", contentInput)
    contentInput = re.sub("#[A-Za-z0-9_]+","", contentInput)
    contentInput = re.sub(r'http\S+', '', contentInput)
    contentInput = re.sub('[()!?]', ' ', contentInput)
    contentInput = re.sub('\[.*?\]',' ', contentInput)
    contentInput = re.sub("[^a-z0-9]"," ", contentInput)
    contentInput = contentInput.strip()
    contentInput = contentInput.split()
    contentInput = [w for w in contentInput if not w in stop]
    contentInput = " ".join(word for word in contentInput)
    return contentInput

stop = stopwords.words('english')

In [18]:
import numpy as np
import plotly.graph_objects as go
import re
from plotly.subplots import make_subplots

In [35]:
df = pd.read_csv("compound_price_data_2.csv")
# figure shared x axes true

fig = make_subplots(specs=[[{"secondary_y": True,}]])

# add trace to figure x as df date y as df price
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price'), secondary_y=False)

# template="plotly_dark"
    
string = "maker.csv"
sentiment_df = pd.read_csv(string)
sentiment_df['date'] = sentiment_df['created'].apply(lambda x: x.split(' ')[0])
sentiment_df.sort_values(['date'], inplace=True)
sentiment_df = sentiment_df.fillna(method='ffill')
sentiment_df['year_month'] = sentiment_df['date'].apply(lambda x: x[0:x.rfind('-')])

monthList = list(sentiment_df['year_month'].unique())
timeSplitData={}
timeSplitDataSentiment = {}

for month in monthList: 
    timeSplitData[month] = []
    timeSplitDataSentiment[month] = []  

def split(x):
    monthData = x['year_month']
    content = x['body']
    cleaned = clean_content(content)
    timeSplitData[monthData].append(cleaned)

sentiment_df.apply(lambda x: split(x), axis=1) 
sid = SentimentIntensityAnalyzer()
for month in timeSplitData:
    for content in timeSplitData[month]:
        output = sid.polarity_scores(content)
        timeSplitDataSentiment[month].append(output)  

timeSplitDataNetScore = {}
for month in monthList:
    timeSplitDataNetScore[month] = 0 
    
for month in timeSplitDataSentiment:
    for output in timeSplitDataSentiment[month]:
        compound = output['compound']
        compound = compound / len(timeSplitDataSentiment[month])
        timeSplitDataNetScore[month] +=compound 

date_dict ={
'2021-01': 'Jan 2021',
'2021-02': 'Feb 2021',
'2021-03': 'Mar 2021',
'2021-04': 'Apr 2021',
'2021-05': 'May 2021',
'2021-06': 'Jun 2021',
'2021-07': 'Jul 2021',
'2021-08': 'Aug 2021',
'2021-09': 'Sep 2021',
'2021-10': 'Oct 2021',
'2021-11': 'Nov 2021',
'2021-12': 'Dec 2021',
}

final = {}
for old_key in timeSplitDataNetScore.keys():
    new_key = date_dict[old_key]
    final[new_key] = timeSplitDataNetScore[old_key]

fig.update_yaxes(autorange="reversed")


fig.add_trace(
go.Scatter(x=list(final.keys()), y=list(final.values()), name="Sentiment"),
secondary_y=True,
)

# fig.update_layout(
#     xaxis = dict(
#         tickmode = "array",
#         tickvals = ['Mar 01, 2022', 'Feb 01, 2022', 'Jan 01, 2022', 'Dec 01, 2021',
#                     'Nov 01, 2021', 'Oct 01, 2021', 'Sep 01, 2021', 'Aug 01, 2021',
#                     'Jul 01, 2021', 'Jun 01, 2021', 'May 01, 2021', 'Apr 01, 2021',
#                     'Mar 01, 2021', 'Feb 01, 2021', 'Jan 01, 2021'],
#         ticktext = ['Mar 2022', 'Feb 2022', 'Jan 2022', 'Dec 2021',
#                     'Nov 2021', 'Oct 2021', 'Sep 2021', 'Aug 2021',
#                     'July 2021', 'June 2021', 'May 2021', 'Apr 2021',
#                     'Mar 2021', 'Feb 2021', 'Jan 2021']
#     )
# )

fig.show()

<ipython-input-9-5076fb747129>:2: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

